In [ ]:
# Please forgive the sloppy programming, this script was written on the fly. 
# Each cell will take some time to load all data from the remote site. Please be patient. 

import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

# Change browser accordingly# browser.get("https://bokmassan.se/for-besokare/program/arets-program/")
#browser = webdriver.Chrome()
browser = webdriver.Firefox()

theelements = []

days = ['https://bokmassan.se/for-besokare/program/?program_date=20160922#program',
       'https://bokmassan.se/for-besokare/program/?program_date=20160923#program',
       'https://bokmassan.se/for-besokare/program/?program_date=20160924#program',
       'https://bokmassan.se/for-besokare/program/?program_date=20160925#program']

# this loop will iterate over all days of the Bokmässan and add the html body to the list theelements. 
for d in days: 
    browser.get(d)
    time.sleep(1)

    elem = browser.find_element_by_tag_name("body")

    no_of_pagedowns = 200

    while no_of_pagedowns:
        elem.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.2)
        no_of_pagedowns-=1

    # Get all the html source, it is easier to barse it with beautifulsoup below
    # Be careful with printing post_elems, the browser might hangup. 
    post_elems = browser.page_source
    theelements.append(post_elems)


In [ ]:
from bs4 import BeautifulSoup
import urllib.request
import re

urllist = []

#this will exctract all links to the individual programpunkter in the schedule and add to the list urllist. 
for t in theelements:
    soup = BeautifulSoup(t, 'html.parser')
    pubitems = soup.findAll({"div" : "permalink"})
    for p in pubitems:
        theurl = re.findall(r'(?<=permalink\=\")(.*)(?=\" title)', str(p), re.IGNORECASE)
        for t in theurl:
            #print(t)
            if t not in urllist:
                urllist.append(t)

In [ ]:
import csv
urlcounter = 0 
programpunktcounter = 1
datalist = []

csvoutputfile = open("programbokmassan.csv", 'w', encoding="utf-8")
fieldnames = ['Typ', 'Tema', 'Deltagare', 'Arrangor']
writer = csv.DictWriter(csvoutputfile, fieldnames=fieldnames, quotechar='"')
writer.writeheader()

for url in urllist:
    urlcounter += 1
    #print("Found URL "+ str(urlcounter) + "\t" + url)
    with urllib.request.urlopen(url) as programpunkt:
        html = programpunkt.read()
        soup = BeautifulSoup(html, 'html.parser')
        detaljer = soup.findAll("div", { "class" : "col-xs-12 col-md-3 programextra__right-column" })
        #print(detaljer[0])
        datalist = detaljer[0].text.split('\n')
        #print(str(len(datalist)))
        #print(datalist)
        
        # This will parse the data in all lenghts of lists. 
        for item in datalist:
            #print(item)
            #print(str(len(item)))
            if "Typ" in datalist:
                typindex = datalist.index("Typ")
                #print(datalist[typindex] + "\t" + datalist[typindex + 1])
                typdata = datalist[typindex + 1]
            else:
                typdata = ""   
            if "Tema" in datalist:
                temaindex = datalist.index("Tema")
                #print(datalist[temaindex] + "\t" + datalist[temaindex + 1])
                temadata = datalist[temaindex + 1]
            else:
                temadata = ""
            if "Deltagare" in datalist:
                deltagareindex = datalist.index("Deltagare")
                #print( datalist[deltagareindex] + "\t" + datalist[deltagareindex + 1])
                deltagaredata = datalist[deltagareindex + 1]
            else:
                deltagaredata = ""
            if "Arrangör" in datalist:
                arrangorindex = datalist.index("Arrangör")
                #print(datalist[arrangorindex] + "\t" + datalist[arrangorindex + 1])
                arrangordata = datalist[arrangorindex + 1]
            else:
                arrangordata = ""
                
    print("Typ: " + typdata)
    print("Tema: " + temadata)
    print("Deltagare: " + deltagaredata)
    print("Arrangör: " + arrangordata)
    writer.writerow({'Typ': typdata, 'Tema': temadata, 'Deltagare': deltagaredata, 'Arrangor': arrangordata})
    print("Added entry " + str(programpunktcounter) + " to csv file.")
    programpunktcounter += 1
    print("---")
csvoutputfile.close()
